In [ ]:
# This instals the Apyori package for using the Association Mining Apriori algorithm
!pip install apyori  
!pip install plotly==5.4.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=14)
mpl.rc('ytick', labelsize=14)


#資料處理套件
import pandas as pd
import numpy as np
import openpyxl
import statistics


#資料視覺化套件:plotly
import plotly.offline
# import plotly.graph_objects as go
import plotly.express as px
# from plotly.subplots import make_subplots
# import plotly.io as pio
# pio.renderers.default='browser'

import matplotlib.pyplot as plt
import seaborn as sns


#方便一次觀察所有
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 匯入時間套件
import datetime as dt

# missingno: 缺失值可視化處理
import missingno as msno

# tqdm: 進度條呈現工具
from tqdm import tqdm

from datetime import datetime

from dateutil.relativedelta import *
import math
import os
import sys
import gdown
import requests

from apyori import apriori


### 載入資料

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/OnlineRetail.csv', encoding='latin1' )

data.shape

# EDA

### 觀察資料

In [ ]:
data.head(5)

print('\n') #間隔兩個資料到呈現

data.tail(5)

print('\n')

data.shape

print('\n')

data.columns

print('\n')

data.describe()

print('\n')

data.info()  # 資料型態均正確

### 缺失資料比例(%)

In [ ]:
df_null = round(100*(data.isnull().sum())/len(data), 2)
df_null

In [ ]:
df_null = data.isnull().sum()
df_null

### 有多少顧客

In [ ]:
plt.rcParams['figure.figsize'] = (35, 7)
x = data.groupby("Country")['CustomerID'].nunique().sort_values(ascending = False)[1:20]

sns.barplot(x = x.index, y = x.values, palette = "flare")

In [ ]:
n = data['CustomerID'].nunique()
n_UK = data.groupby("Country")['CustomerID'].nunique().sort_values(ascending = False)[:1].sum()

print("There are {} different customers in total.".format(n))
print("There are {} different customers in UK.".format(n_UK))
print("There are {} different customers in other countries.".format(n-n_UK))



## 處理缺失的 customerID 問題



### 總顧客數及各國顧客數

In [ ]:
customerID_in_everycountry = data.groupby("Country")['CustomerID'].nunique().sort_values(ascending = False)
print(customerID_in_everycountry[:10])
print("\nThere are {} different customers in total.".format(customerID_in_everycountry.sum()))

### 全部不同的 invoiceNo 數



In [ ]:
invoiceNo_in_everycountry = data.groupby("Country")['InvoiceNo'].nunique().sort_values(ascending = False)
print(invoiceNo_in_everycountry[:10])   # 僅印出10筆
print("\nWe have {} different invoiceNo in total.".format(invoiceNo_in_everycountry.sum()))

### 找缺失的cus_id 中，不重複的發票號碼

In [ ]:
data_fillzero = data.fillna({"CustomerID":0})
data_0 = data_fillzero[data_fillzero.loc[:,"CustomerID"]==0.0]
memberID_lost = data_0.groupby("Country")["InvoiceNo"].nunique().sort_values(ascending = False)

print(memberID_lost)
print("\nThere are {} different invoiceNo losted.".format(memberID_lost.sum()))

列出有缺顧客ID國家的缺失占比(by InvoiceID)：

*   All               3710/25900 = **0.14**
*   United Kingdom    3637/23494 = **0.15**
*   EIRE                  41/360 = **0.11**
*   Hong Kong              15/15 = **1**
*   Unspecified             5/13 = **0.38**
*   Switzerland             3/74 = **0.04**
*   Israel                   3/9 = **0.33**
*   France                 3/461 = **0.01**
*   Bahrain                  2/4 = **0.50**
*   Portugal                1/71 = **0.01**




In [ ]:
# 寫法二
# a = data[data.loc[:,"CustomerID"].isnull().values == True]
# data_lostID = a.groupby("Country")["InvoiceNo"].nunique().sort_values(ascending = False)
# data_lostID

In [ ]:
# 寫法三
# b = data[data.isnull().T.any()]
# data_lostID = b.groupby("Country")["InvoiceNo"].nunique().sort_values(ascending = False)
# data_lostID

## 資料視覺化

### 單價分布

In [ ]:
plt.rcParams['figure.figsize'] = (15, 7)
plt.style.use( 'seaborn-whitegrid')
sns.displot(data["UnitPrice"][:100], color = "green")  # 太長跑不動

plt.show()


In [ ]:
data['Country'].value_counts().head(20)

### 每國交易量

In [ ]:
plt.rcParams['figure.figsize'] = (6, 5)
a = data['Country'].value_counts()[1:8]
sns.barplot(x = a.values, y = a.index, palette = "flare")
plt.title('Top 20 Countries having Online Retail Market', fontsize = 20)
plt.xlabel('Count')
plt.ylabel('Names of Countries')
plt.show()

In [ ]:
data['Country'].value_counts().sum()

In [ ]:
data['Country'].value_counts()

In [ ]:
plt.rcParams['figure.figsize'] = (12, 10)
a = data['Country'].value_counts().tail(20)
sns.barplot(x = a.values, y = a.index, palette = "flare")
plt.title('Bottom 20 Countries having Online Retail Market', fontsize = 20)
plt.xlabel('Names of Countries')
plt.ylabel('Count')
plt.show()

### 各國商品銷售數量

In [ ]:
plt.rcParams['figure.figsize'] = (12, 10)

a = data['Quantity'].groupby(data['Country']).agg('sum').sort_values(ascending = False)[1:]
print(a)

sns.barplot(x = a.values, y = a.index, palette = 'flare')
plt.title('Quantity of Products sold in all the countries except UK')
plt.show()

In [ ]:
#color = plt.cm.viridis(np.linspace(0, 1, 20))
data['Quantity'].groupby(data['Country']).agg('sum').sort_values(ascending = False).tail(20).plot.bar(figsize = (15, 7), color = "lightblue")

plt.title('Bottom 20 Countries according to Quantity Sold Online', fontsize = 20)
plt.xlabel('Names of the Countries')
plt.ylabel('Number of Items Sold')
plt.show()

### 商品描述出現最多的字

In [ ]:
from wordcloud import WordCloud
from wordcloud import STOPWORDS

stopwords = set(STOPWORDS)
wordcloud = WordCloud(background_color = 'white', width = 900, height = 900).generate(str(data['Description']))

print(wordcloud)
plt.rcParams['figure.figsize'] = (12, 12)
plt.axis('off')
plt.imshow(wordcloud)
plt.title('Most Occuring word in the Description list', fontsize = 20)
plt.show()

### 去掉退貨的部分

In [ ]:
data_want = data[data.loc[:,"Quantity"]>0]
data_want.info()

### 增加銷售額欄位

In [ ]:
data_want["Sales"] = data_want["UnitPrice"]*data_want["Quantity"]
data_want

### 銷售額分布

In [ ]:
plt.rcParams['figure.figsize'] = (25, 5)
sns.displot(data_want['Sales'][1:100], color = 'crimson')           # 太長跑不動 
plt.title('Distribution of Sales in entire globe', fontsize = 20)
plt.xlabel('Sales in different Countries')
plt.ylabel('Sales')
plt.show()

### 各國銷售狀況

In [ ]:
plt.rcParams['figure.figsize'] = (9, 12)
a = data_want['Sales'].groupby(data_want['Country']).agg('sum').sort_values(ascending = False)[1:]

print(a)

sns.barplot(x = a.values, y = a.index, palette = 'icefire_r')
plt.title('Sales of all the Countries Except UK')

plt.show()

### 後20名銷售狀況

In [ ]:
data_want['Sales'].groupby(data_want['Country']).agg('sum').sort_values(ascending = False).tail(20).plot.bar(figsize = (15, 7), color = 'lightblue')
plt.title('Bottom 20 Countries Sales', fontsize = 20)
plt.xlabel('Names of Countries')
plt.ylabel('Sales')
plt.show()

### 時間序列分析(銷售額)

In [ ]:
plt.rcParams['figure.figsize'] = (30, 5)

data_want.plot(x = 'InvoiceDate', y = 'Sales')
plt.title("Time Series Analysis of Sales", fontsize = 20)
plt.xlabel('Date of Purchase')
plt.ylabel('Sales')
plt.show()

### 時間序列分析(各國銷售額)

In [ ]:
def time_series(country, xlabel='InvoiceDate', dpi=100):
  # plt.figure(figsize=(30,5))
  dataset = data_want[data_want["Country"]== country ]
  dataset.plot(x = "InvoiceDate", y = "Sales")
  plt.title("Time-Series plot for {}".format(country),fontsize = 20)
  plt.legend(loc="upper left", frameon = True, fontsize=20 )


In [ ]:
plt.rcParams['figure.figsize'] = (30, 5)

time_series("EIRE")
time_series("Germany")
time_series("France")

## 熱銷商品分析❌


讀進檔案

In [ ]:
data = pd.read_csv(data_dir + "OnlineRetail.csv", encoding='latin1' )
data_want = data[data.loc[:,"Quantity"]>0]
data_want["Sales"] = data_want["UnitPrice"]*data_want["Quantity"]

In [ ]:
# we got different UnitPrice for one product
# need a new column of avg_unit_price

avg_unit_price_test = data_want[data_want.loc[:, 'Description']=='JUMBO BAG RED RETROSPOT']
avg_unit_price_test['UnitPrice'].value_counts()

In [ ]:
# 產生top_seller資料集

top_seller = data_want[['Description', 'Quantity', 'Sales']].groupby('Description',as_index=False).sum()
top_seller['avg_Unit_Price']= top_seller['Sales']/top_seller['Quantity']
top_seller.head()

In [ ]:
# top seller by Sales
Sales_bar = top_seller.sort_values(by=['Sales'], ascending=False).head(50)
Sales_bar.shape
print('\n')
# top seller by Quantity
Quantity_bar = top_seller.sort_values(by=['Quantity'], ascending=False).head(50)
Quantity_bar.shape
print('\n')
# top seller by avg_Unit_Price
avg_Unit_Price_bar = top_seller.sort_values(by=['avg_Unit_Price'], ascending=False).head(50)
avg_Unit_Price_bar.shape

有三種 hot_item

In [ ]:
hot_item_quantity = Quantity_bar
hot_item_list_quantity = pd.Series.to_list(hot_item_quantity['Description']) 
len(hot_item_list_quantity)

print('\n=====')

hot_item_sales = Sales_bar
hot_item_list_sales = pd.Series.to_list(hot_item_sales['Description']) 
len(hot_item_list_sales)

print('\n=====')

hot_item_avg_Unit_Price = avg_Unit_Price_bar
hot_item_list_avg_Unit_Price = pd.Series.to_list(hot_item_avg_Unit_Price['Description']) 
len(hot_item_avg_Unit_Price)

list 加上[ ]

In [ ]:
hot_item_list_done_quantity =[]
for i in range(len(hot_item_list_quantity)):
  x ='[' + str(hot_item_list_quantity[i]) + ']'
  hot_item_list_done_quantity.append(x)

len(hot_item_list_done_quantity)

print('\n=====')

hot_item_list_done_sales =[]
for i in range(len(hot_item_list_sales)):
  x ='[' + str(hot_item_list_sales[i]) + ']'
  hot_item_list_done_sales.append(x)

len(hot_item_list_done_sales)

print('\n=====')

hot_item_list_done_avg_Unit_Price =[]
for i in range(len(hot_item_avg_Unit_Price)):
  x ='[' + str(hot_item_list_avg_Unit_Price[i]) + ']' 
  hot_item_list_done_avg_Unit_Price.append(x)

len(hot_item_list_done_avg_Unit_Price)


In [ ]:
hot_item_list_done_quantity
hot_item_list_done_sales
hot_item_list_done_avg_Unit_Price

# data_UK EDA(用字串擷取)

增加銷售額 去掉退貨

In [ ]:
data_want = data[data.loc[:,"Quantity"]>0]
data_want["Sales"] = data_want["UnitPrice"]*data_want["Quantity"]
data_want = data_want

In [ ]:
data_uk = data_want[data_want['Country'] == 'United Kingdom']
data_uk.info()

增加小時、月份欄位

In [ ]:
gethour=[]
hours = data_uk['InvoiceDate']
for hour in hours:
  i = hour[11:13]
  gethour.append(i)

data_uk['Hours'] = gethour

data_uk

In [ ]:
getMonth=[]
Months = data_uk['InvoiceDate']
for Month in Months:
  a = Month[3:5]
  b = Month[6:11]
  getMonth.append(b+'-'+a)

data_uk['Months'] = getMonth
data_uk

時間序列分析

In [ ]:
# 時間格式轉換
# from datetime import datetime, timedelta
# data_uk['InvoiceDate'] = pd.to_datetime(data_want['InvoiceDate'])
# data_uk.info()

In [ ]:
# 每月銷售量
M = data_uk.groupby('Months')['Sales'].sum()
xtick = [0,1,2,3,4,5,6,7,8,9,10,11,12]
M.plot(kind= 'line',rot =0 , fontsize=12, figsize=(15,5), grid=True,color='green', linestyle='solid', marker='o',
     markerfacecolor='black', markersize=8, xticks=xtick )

熱力圖

In [ ]:
# 取出dataframe
Sales_by_momth = pd.DataFrame(data=data_uk.loc[:, ['Months','Sales']].groupby('Months')['Sales'].sum().astype(int))
Sales_by_momth

In [ ]:
# 畫熱力圖
plt.figure(figsize = (15,10))
sns.heatmap(data=Sales_by_momth, cmap='flare', annot= True, fmt='d', linewidths=0.3)
plt.title('heatmap of Sales by Months',fontsize=20)

In [ ]:
# 不同時段銷量
T = data_uk.groupby('Hours')['Sales'].sum()
T.plot(kind= 'bar',rot =0, fontsize=20, figsize=(15,5),grid=True)

In [ ]:
# 取出dataframe
Sales_by_Hours = pd.DataFrame(data=data_uk.loc[:, ['Hours','Sales']].groupby('Hours')['Sales'].sum().astype(int))
Sales_by_Hours

In [ ]:
plt.figure(figsize = (15,10))
sns.heatmap(data=Sales_by_Hours, cmap='flare', annot= True, fmt='d', linewidths=0.3)
plt.title('heatmap of Sales by Hours',fontsize=20)

In [ ]:
data_uk_hm = data_uk[['Hours','Sales','Months']]
data_uk_hm_df = data_uk_hm.pivot_table(index='Months',columns='Hours', values='Sales',aggfunc='mean')
data_uk_hm_df.head()

In [ ]:
plt.figure(figsize = (10,5))
sns.heatmap(data_uk_hm_df, annot=True, fmt='.1f',linewidths=0.5,cmap='YlOrRd')

plt.title('Heatmap of Hours and Months', fontsize=15)

# data_UK EDA(time series)

增加銷售額 去掉退貨

In [ ]:
data_want = data[data.loc[:,"Quantity"]>0]
data_want["Sales"] = data_want["UnitPrice"]*data_want["Quantity"]
data_want["Sales"] =data_want["Sales"].astype(int)
data_want

In [ ]:
data_uk = data_want[data_want['Country'] == 'United Kingdom']
data_uk.head()

轉換時間格式

In [ ]:
# transform data to datetime object

data_uk.InvoiceDate= pd.to_datetime(data_uk.InvoiceDate)
data_uk.info()


[map the time](https://www.runoob.com/python/att-time-strftime.html)

In [ ]:
# data_uk['InvoiceDate'] = pd.to_datetime(data_uk['InvoiceDate'])
# data_uk['datetime'] = data_uk['InvoiceDate'].dt.strftime('%Y %B %d %H' )

# data_uk['Year'] = data_uk['datetime'].map(lambda x:x. split(' ')[0])
# data_uk['Hours'] = data_uk['datetime'].map(lambda x:x. split(' ')[-1])
# data_uk['Day'] = data_uk['datetime'].map(lambda x:x. split(' ')[-2])
# data_uk['Months'] = data_uk['datetime'].map(lambda x:x. split(' ')[-3])

### 增加欄位

In [ ]:
# add a hour column

# define the bins
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

# add custom labels if desired
labels = ['00:00-00:59', '01:00-01:59', '02:00-02:59', '03:00-03:59', '04:00-04:59', '05:00-05:59', '06:00-06:59', '07:00-07:59',
          '08:00-08:59', '09:00-09:59', '10:00-10:59', '11:00-11:59', '12:00-12:59', '13:00-13:59', '14:00-14:59', '15:00-15:59', 
          '16:00-16:59', '17:00-17:59', '18:00-18:59', '19:00-19:59', '20:00-20:59', '21:00-21:59', '22:00-22:59', '23:00-23:59']

# add the bins to the dataframe
data_uk['Time'] = pd.cut(data_uk.InvoiceDate.dt.hour, bins, labels=labels, right=False)

In [ ]:
# add a month column

bins = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,12,13]

# add custom labels if desired
labels = ['01', '02', '03','04','05','06','07','08','09', '10', '11', '12']

# add the bins to the dataframe
data_uk['Month'] = pd.cut(data_uk.InvoiceDate.dt.month, bins, labels=labels, right=False)

In [ ]:
# add a year column

bins = [2010, 2011, 2012]

# add custom labels if desired
labels = ['2010', '2011']

# add the bins to the dataframe
data_uk['Year'] = pd.cut(data_uk.InvoiceDate.dt.year, bins, labels=labels, right=False)
data_uk

In [ ]:
# add a year&month column

data_uk['Year&month'] = pd.to_datetime(data_uk['Year'].astype(str) +' '+ data_uk['Month'].astype(str))

### 畫圖

In [ ]:
# groupby time bin ,draw bar plot

data_uk_g = data_uk[['Time', 'Sales']].groupby('Time', as_index=False)['Sales'].sum()

px.bar(data_uk_g,x='Time',y=data_uk_g['Sales'], text='Sales',title='英國不同時段銷售量比較', color='Time',color_discrete_sequence=px.colors.qualitative.Pastel) 

In [ ]:
# groupby year & month bin ,draw line plot

data_uk_g = data_uk[['Year&month', 'Sales']].groupby(['Year&month'], as_index=False)['Sales'].sum()
data_uk_g
px.line(data_uk_g,x='Year&month',y='Sales', text='Sales',title='英國不同月份銷售量變化') 

In [ ]:
# groupby InvoiceDate ,draw line plot

data_uk_g = data_uk[['InvoiceDate', 'Sales']].groupby(['InvoiceDate'], as_index=False)['Sales'].sum()
data_uk_g
px.line(data_uk_g,x='InvoiceDate',y='Sales') 

In [ ]:
data_uk_timeseries = data_uk.drop(['Description','InvoiceNo', 'StockCode','Quantity','UnitPrice','CustomerID'], axis=1)
data_uk_timeseries

In [ ]:
# data_uk_timeseries.to_csv("data_uk_timeseries.csv", index=False)

# RFM analysis


## 資料準備

### 資料整理

In [ ]:
import datetime as dt


In [ ]:
data.info()

In [ ]:
# 移除缺失值(理由見上方EDA)

data.dropna(inplace = True)
data.info()

In [ ]:
#調整 CustomerID、InvoiceDate 的資料格式

data["CustomerID"] = data["CustomerID"].astype(str)
data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"],format='%d-%m-%Y %H:%M')

data.info()

In [ ]:
data.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

In [ ]:
# 另一種寫法
# data1 = data[~data["InvoiceNo"].str.contains("C", na=False)]
# data1.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

In [ ]:
# 移除 Quantity	<0

data_describe = data[data.loc[:,"Quantity"]>0]
data_describe.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

### R、F、M資料準備

In [ ]:
#M
data["Amount"] = data["Quantity"] * data["UnitPrice"]
rfm_m = data.groupby("CustomerID")["Amount"].sum()
rfm_m = rfm_m.reset_index()
rfm_m.head()
rfm_m.shape

In [ ]:
#F

rfm_f = data.groupby("CustomerID")["InvoiceNo"].count()
rfm_f = rfm_f.reset_index()
rfm_f.columns = ["CustomerID", "Frequency"]

rfm_f.shape

In [ ]:
# R
data["InvoiceDate"].max()

In [ ]:
# 設定比較日為最後交易日+1日，以免R出現0
set_date = data["InvoiceDate"].max() + dt.timedelta(days = 1)
set_date

In [ ]:
# 計算交易日差
rfm_r = (set_date - data.groupby("CustomerID")["InvoiceDate"].max()).dt.days
rfm_r = rfm_r.reset_index()
rfm_r.columns = ["CustomerID", "Diff"]

rfm_r.head()

In [ ]:
country_data = data.loc[:, ["CustomerID", "Country"]]
country_data

In [ ]:
#合併RFM
# pd.merge 一次只能合併兩個，要分兩次合併 https://stackoverflow.com/questions/46008957/pandas-merge-df-error


rfm = pd.merge(rfm_r, rfm_f, on="CustomerID", how="inner")
rfm = pd.merge(rfm, rfm_m, on="CustomerID", how="inner")


rfm.columns=["CustomerID","Recency", "Frequency", "Monetary"]

rfm

In [ ]:
# 增加RFM+Country

RFM_Country = pd.merge(rfm, country_data, how="inner", on=["CustomerID"])

RFM_Country

In [ ]:
# 移除重複的列
RFM_Country.drop_duplicates(keep='first',inplace=True)

RFM_Country

In [ ]:
# 找出重複的ID(即一個ID有兩個國家消費紀錄)
Count =RFM_Country.groupby("CustomerID")["CustomerID"].count()
RFM_Count = pd.DataFrame(data=Count,columns= ["CustomerID"])#.reset_index(inplace=True)

twice_customer = RFM_Count[RFM_Count["CustomerID"] > 1]
twice_customer

In [ ]:
# 下載資料觀察

# RFM_Country.to_csv("RFM_country.csv", index=False)
# twice_customer.to_csv("twice_customer.csv", index=False)

## 整體RFM資料觀察

In [ ]:
# 下載資料觀察

rfm.to_csv("rfm.csv", index=False)

rfm.head()

In [ ]:
rfm.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

## 針對英國做RFM(標準化)

### 取出RFM_UK，觀察分布狀況

In [ ]:
# 取出RFM_UK資料集，只留下國家跟RFM

rfm_UK = RFM_Country[RFM_Country.loc[:, "Country"] == "United Kingdom"]#.drop("CustomerID", axis = 1)
rfm_UK

In [ ]:
# 英國的RFM分布狀況

attributes = ['Monetary','Frequency','Recency']
plt.rcParams['figure.figsize'] = [10,8]
sns.boxplot(data = rfm_UK[attributes], orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
plt.title("Outliers Variable Distribution of UK market", fontsize = 14, fontweight = 'bold')
plt.ylabel("Range", fontweight = 'bold')
plt.xlabel("Attributes", fontweight = 'bold')

### 移除RFM_UK離群值

In [ ]:
# 移除英國的RFM離群值(rfm_UK已經去掉離群值)

# Removing (statistical) outliers for Monetary
Q1 = rfm_UK.Monetary.quantile(0.05)
Q3 = rfm_UK.Monetary.quantile(0.95)
IQR = Q3 - Q1
rfm_UK = rfm_UK[(rfm_UK.Monetary >= Q1 - 1.5*IQR) & (rfm_UK.Monetary <= Q3 + 1.5*IQR)]

# Removing (statistical) outliers for Recency
Q1 = rfm_UK.Recency.quantile(0.05)
Q3 = rfm_UK.Recency.quantile(0.95)
IQR = Q3 - Q1
rfm_UK = rfm_UK[(rfm_UK.Recency >= Q1 - 1.5*IQR) & (rfm_UK.Recency <= Q3 + 1.5*IQR)]

# Removing (statistical) outliers for Frequency
Q1 = rfm_UK.Frequency.quantile(0.05)
Q3 = rfm_UK.Frequency.quantile(0.95)
IQR = Q3 - Q1
rfm_UK = rfm_UK[(rfm_UK.Frequency >= Q1 - 1.5*IQR) & (rfm_UK.Frequency <= Q3 + 1.5*IQR)]

# rfm_UK_clusters = rfm_UK.drop("Country", axis = 1).drop("CustomerID", axis = 1)
# rfm_UK_clusters

標準化

In [ ]:
from sklearn.preprocessing import StandardScaler 

rfm_UK_std = rfm_UK[["Monetary", "Frequency", "Recency"]]

scaler = StandardScaler()

rfm_UK_scaled = scaler.fit_transform(rfm_UK_std)
rfm_UK_scaled.shape

### 分群

將採用**K-means演算法**分群，先用**輪廓係數法Silhouette analysis**

*   輪廓係數法：**找出同群資料點內最近/不同群越分散**的值，也就是滿足 Cluster 的定義
*   ***S=(b-a)/max(a,b)***

*  b為不同群之間的點平均距離，a為同群之間的距離，S則越大越好，代表分得越清楚
*   資料來源 https://reurl.cc/95ANoO
*   資料來源 https://reurl.cc/1oLA7D









In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

手肘法（ElbowMethod）
群內的總變異(SSE)會最小

In [ ]:
# Elbow-curve/SSE

sse = []
for num_clusters in range(2,11):
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(rfm_UK_scaled)
    
    sse.append(kmeans.inertia_)
    
# plot the SSDs for each n_clusters

X = range(2, 11)
plt.xlabel("number of clusters, k")
plt.ylabel("SSE")
plt.plot(X, sse, 'o-', color='black')
plt.show()

手肘法（程式2）

In [ ]:
#Implementing the Elbow method to identify the ideal value of 'k'. 

ks = range(1,10) #hit and trial, let's try it 10 times.
inertias = []
for k in ks:
    model = KMeans(n_clusters=k)    # Create a KMeans instance with k clusters: model
    model.fit(rfm_UK_scaled)                    # Fit model to samples
    inertias.append(model.inertia_) # Append the inertia to the list of inertias
    
plt.plot(ks, inertias, '-o', color='black') #Plotting. The plot will give the 'elbow'.
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

輪廓係數法

In [ ]:
# Silhouette analysis
silhouette_avg_for_plot =[]
#range_n_clusters = [2, 3, 4, 5]

for num_clusters in range(2,11):
    
    # intialise kmeans
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(rfm_UK_scaled)
    
    cluster_labels = kmeans.labels_
    
    # silhouette score
    silhouette_avg = silhouette_score(rfm_UK_scaled, cluster_labels)
    silhouette_avg_for_plot.append(silhouette_score(rfm_UK_scaled, cluster_labels))
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg)) 

plt.plot(range(2,11), silhouette_avg_for_plot)


選擇分成三群


### 分群結果

In [ ]:
kmeans = KMeans(n_clusters=3, max_iter=50)
kmeans.fit(rfm_UK_scaled)

In [ ]:
kmeans.labels_

In [ ]:
# 加上分群結果(rfm_UK已經加入分群結果欄位)

rfm_UK["Cluster_Id"] = kmeans.labels_
rfm_UK.head()

In [ ]:
corr = rfm_UK.corr()
plt.subplots(figsize=(15,10))

sns.heatmap(
    corr, 
    xticklabels=corr.columns,  # x標籤 
    yticklabels=corr.columns,  # y標籤
    annot = True ,              # 註解
    cmap = 'coolwarm'
    )

下載檔案觀察

In [ ]:
rfm_UK.to_csv("rfm_UK.csv", index=False)

In [ ]:
rfm_UK.to_excel("rfm_UK.xlsx", index=False)

畫出圖形觀察

In [ ]:
fig = plt.figure(num = 1, figsize = [25, 7])

plt.subplot(1, 3, 1)
sns.boxplot(x="Cluster_Id", y="Recency", data=rfm_UK)
plt.title("Cluster_Id vs. Recency", fontsize = 14, fontweight = 'bold')
plt.ylabel("Recency", fontweight = 'bold')
plt.xlabel("Cluster_Id", fontweight = 'bold')

plt.subplot(1, 3, 2)
sns.boxplot(x="Cluster_Id", y="Frequency", data=rfm_UK)
plt.title("Cluster_Id vs. Frequency", fontsize = 14, fontweight = 'bold')
plt.ylabel("Frequency", fontweight = 'bold')
plt.xlabel("Cluster_Id", fontweight = 'bold')

plt.subplot(1, 3, 3)
sns.boxplot(x="Cluster_Id", y="Monetary", data=rfm_UK)
plt.title("Cluster_Id vs. Monetary", fontsize = 14, fontweight = 'bold')
plt.ylabel("Monetary", fontweight = 'bold')
plt.xlabel("Cluster_Id", fontweight = 'bold')

fig.savefig('Cluster_by_Kmeans.png')


In [ ]:
rfm_UK_0 = rfm_UK[rfm_UK.loc[:, "Cluster_Id"] ==0]
rfm_UK_1 = rfm_UK[rfm_UK.loc[:, "Cluster_Id"] ==1]
rfm_UK_2 = rfm_UK[rfm_UK.loc[:, "Cluster_Id"] ==2]

rfm_UK_0.describe()

In [ ]:
# 檢查分布
# plt.rcParams['figure.figsize'] = (15, 7)
# plt.style.use( 'seaborn-whitegrid')
# sns.displot(rfm_UK_0["Recency"], color = "green")  # 太長跑不動

# plt.show()


In [ ]:
rfm_UK_1.describe()

In [ ]:
rfm_UK_2.describe()

## 針對其他國家做RFM(標準化)

### 取出RFM_other，觀察分布狀況

In [ ]:
# 取出RFM_UK資料集，只留下國家跟RFM

rfm_other = RFM_Country[RFM_Country.loc[:, "Country"] != "United Kingdom"]#.drop("CustomerID", axis = 1)
rfm_other

In [ ]:
# 其他國家的RFM分布狀況

attributes = ['Monetary','Frequency','Recency']
plt.rcParams['figure.figsize'] = [10,8]
sns.boxplot(data = rfm_other[attributes], orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
plt.title("Outliers Variable Distribution of overseas market", fontsize = 14, fontweight = 'bold')
plt.ylabel("Range", fontweight = 'bold')
plt.xlabel("Attributes", fontweight = 'bold')

### 移除rfm_other離群值

In [ ]:
# 移除其他國家的RFM離群值(rfm_other已經去掉離群值)

# Removing (statistical) outliers for Amount
Q1 = rfm_other.Monetary.quantile(0.05)
Q3 = rfm_other.Monetary.quantile(0.95)
IQR = Q3 - Q1
rfm_other = rfm_other[(rfm_other.Monetary >= Q1 - 1.5*IQR) & (rfm_other.Monetary <= Q3 + 1.5*IQR)]

# Removing (statistical) outliers for Recency
Q1 = rfm_other.Recency.quantile(0.05)
Q3 = rfm_other.Recency.quantile(0.95)
IQR = Q3 - Q1
rfm_other = rfm_other[(rfm_other.Recency >= Q1 - 1.5*IQR) & (rfm_other.Recency <= Q3 + 1.5*IQR)]

# Removing (statistical) outliers for Frequency
Q1 = rfm_other.Frequency.quantile(0.05)
Q3 = rfm_other.Frequency.quantile(0.95)
IQR = Q3 - Q1
rfm_other = rfm_other[(rfm_other.Frequency >= Q1 - 1.5*IQR) & (rfm_other.Frequency <= Q3 + 1.5*IQR)]
rfm_other

In [ ]:
# 標準化
from sklearn.preprocessing import StandardScaler 

rfm_other_std = rfm_other[["Monetary", "Frequency", "Recency"]]

scaler = StandardScaler()

rfm_other_scaled = scaler.fit_transform(rfm_other_std)
rfm_other_scaled.shape

### 分群

將採用**K-means演算法**分群，先用**輪廓係數法Silhouette analysis**

*   輪廓係數法：**找出同群資料點內最近/不同群越分散**的值，也就是滿足 Cluster 的定義
*   ***S=(b-a)/max(a,b)***

*  b為不同群之間的點平均距離，a為同群之間的距離，S則越大越好，代表分得越清楚
*   資料來源 https://reurl.cc/95ANoO
*   資料來源 https://reurl.cc/1oLA7D









In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

手肘法（ElbowMethod）

In [ ]:
# Elbow-curve/SSE

sse = []
for num_clusters in range(2,11):
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(rfm_other_scaled)
    
    sse.append(kmeans.inertia_)
    
# plot the SSDs for each n_clusters

X = range(2, 11)
plt.xlabel("num_clusters")
plt.ylabel("SSE")
plt.plot(X, sse, 'o-')
plt.show()

輪廓係數法

In [ ]:
# Silhouette analysis
silhouette_avg_for_plot =[]
#range_n_clusters = [2, 3, 4, 5]

for num_clusters in range(2,11):
    
    # intialise kmeans
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(rfm_other_scaled)
    
    cluster_labels = kmeans.labels_
    
    # silhouette score
    silhouette_avg = silhouette_score(rfm_other_scaled, cluster_labels)
    silhouette_avg_for_plot.append(silhouette_score(rfm_other_scaled, cluster_labels))
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg)) 

plt.plot(range(2,11), silhouette_avg_for_plot)


選擇分成三群


### 分群結果

In [ ]:
kmeans = KMeans(n_clusters=3, max_iter=50)
kmeans.fit(rfm_other_clusters)

In [ ]:
kmeans.labels_

In [ ]:
# 加上分群結果(rfm_UK已經加入分群結果欄位)

rfm_other["Cluster_Id"] = kmeans.labels_
rfm_other.head()

下載檔案觀察

In [ ]:
# rfm_other.to_csv("rfm_other.csv", index=False)

畫出圖形觀察

In [ ]:
fig = plt.figure(num = 1, figsize = [25, 7])

plt.subplot(1, 3, 1)
sns.boxplot(x="Cluster_Id", y="Recency", data=rfm_other)
plt.title("Cluster_Id vs. Recency", fontsize = 14, fontweight = 'bold')
plt.ylabel("Recency", fontweight = 'bold')
plt.xlabel("Cluster_Id", fontweight = 'bold')

plt.subplot(1, 3, 2)
sns.boxplot(x="Cluster_Id", y="Frequency", data=rfm_other)
plt.title("Cluster_Id vs. Frequency", fontsize = 14, fontweight = 'bold')
plt.ylabel("Frequency", fontweight = 'bold')
plt.xlabel("Cluster_Id", fontweight = 'bold')

plt.subplot(1, 3, 3)
sns.boxplot(x="Cluster_Id", y="Monetary", data=rfm_other)
plt.title("Cluster_Id vs. Monetary", fontsize = 14, fontweight = 'bold')
plt.ylabel("Monetary", fontweight = 'bold')
plt.xlabel("Cluster_Id", fontweight = 'bold')

fig.savefig('Cluster_by_Kmeans.png')


In [ ]:
rfm_other_0 = rfm_other[rfm_other.loc[:, "Cluster_Id"] ==0]
rfm_other_1 = rfm_other[rfm_other.loc[:, "Cluster_Id"] ==1]
rfm_other_2 = rfm_other[rfm_other.loc[:, "Cluster_Id"] ==2]

rfm_other_0.describe()

In [ ]:
rfm_other_1.describe()

In [ ]:
rfm_other_2.describe()

# Apriori Analysis

## 安裝套件及載入資料

In [ ]:
order_products = pd.read_csv("/content/drive/MyDrive/OnlineRetail.csv", encoding='latin1')
order_products = order_products[(order_products["Quantity"]>0)&(order_products["UnitPrice"]>0)]

order_products

##檢查載入資料
一個 StockCode  對應不只一個 Description，需決定一種

In [ ]:
# 一個 StockCode  對應不只一個 Description

len(order_products['StockCode'].unique())
print('\n')
len(order_products['Description'].unique())

In [ ]:
order_products['InvoiceNo'].value_counts().index

### 用InvoiceNo找關連規則(這次不用)

In [ ]:
# record_1=[]
# for i in tqdm(order_products['InvoiceNo'].value_counts().index):                    # i = 所有的InvoiceNo
#     member = order_products[order_products['InvoiceNo']==i]                         # each member list
#     record_1.append(member['StockCode'].values.tolist())                              # product list for every member
# association_rules_1 = apriori(record_1, min_support=0.01,min_confidence=0.6, min_lift=1.0000001)
# association_results_1 = list(association_rules_1)

In [ ]:
# association_DF_1 = pd.DataFrame(association_results_1)
# association_DF_1

### 建立產品列表，新增product_name欄位

In [ ]:
a = list(order_products['Description'])
len(a)

In [ ]:
product_name=[]
for i in range(len(a)):
  c = '[' + str(a[i]) +']'
  product_name.append(c)

len(product_name)

In [ ]:
order_products['product_name']=product_name
order_products.head()

## 跑關聯規則

In [ ]:
record=[]
for i in tqdm(order_products['InvoiceNo'].value_counts().index):
    member = order_products[order_products['InvoiceNo']==i]
    record.append(member['product_name'].values.tolist())   
association_rules = apriori(record, min_support=0.01, min_confidence=0.5, min_lift=1.0000001)
association_results = list(association_rules)

In [ ]:
association_results

In [ ]:
association_DF = pd.DataFrame(association_results)
# association_DF.to_csv('association_DF.csv', index=False)
association_DF

##建立Dataframe
[參考資料](https://artsdatascience.wordpress.com/2019/12/13/python-%E5%AF%A6%E6%88%B0%E7%AF%87%EF%BC%9A%E7%94%A8apyori-library%E5%8E%BB%E5%81%9A-apriori-algorithm/)

In [ ]:
association_DF

In [ ]:
item_base= []
item_add= []
support= []
Confidence= []
Lift= []

for item in range(len(association_DF['ordered_statistics'])):
  for j in range(len(association_DF['ordered_statistics'][item])):
    item_base.append(association_DF['ordered_statistics'][item][j][0])
    item_add.append(association_DF['ordered_statistics'][item][j][1])
    support.append(association_DF['support'][item])
    Confidence.append(association_DF['ordered_statistics'][item][j][2]) 
    Lift.append(association_DF['ordered_statistics'][item][j][3])


In [ ]:
dict_for_df ={
    'product_A' : item_base,
    'product_B' : item_add,
    'Support' : support,
    'Confidence' : Confidence,
    'Lift' : Lift
}

association_rules_df = pd.DataFrame(dict_for_df)
association_rules_df

In [ ]:
# association_rules_df.to_csv('association_rules_df.csv', index=False)

In [ ]:
# 我的方法
# item_base= []
# item_add= []
# support= []
# Confidence= []
# Lift= []

# for item in association_results:
#   pair = item[0]
#   items = [x for x in pair]
#   item_base.append(items[0])
#   item_add.append(items[1])
#   support.append(float(item[1]))
#   Confidence.append(float(item[2][0][2]))
#   Lift.append(float(item[2][0][3]))

In [ ]:
# dict_for_df ={
#     'product_A' : item_base,
#     'product_B' : item_add,
#     'Support' : support,
#     'Confidence' : Confidence,
#     'Lift' : Lift
# }

# association_rules_df = pd.DataFrame(dict_for_df)
# association_rules_df

In [ ]:
# remove [ ](先不用)
# aaa = association_rules_df['product_A']
# Product_A=[]
# for i in range(len(aaa)):
#   s = str(aaa[i])[1:-1]
#   Product_A.append(s)

# bbb = association_rules_df['product_B']
# Product_B=[]
# for i in range(len(aaa)):
#   t = str(bbb[i])[1:-1]
#   Product_B.append(t)

# len(Product_A)
# print('\n')
# len(Product_B)



In [ ]:
# replace product_A & B columns(先不用)

# dict_for_removement ={
#     'Product_A' : Product_A,
#     'Product_B' : Product_B,
#     'Support' : support,
#     'Confidence' : Confidence,
#     'Lift' : Lift
# }

# association_rules_df = pd.DataFrame(dict_for_removement)
# association_rules_df

In [ ]:
# association_rules_df.to_csv("association_rules_df.csv", index=False)

## 檢查關聯規則資料

In [ ]:
# association_rules_df = pd.read_csv('/content/drive/MyDrive/online retailing/input/association_rules_df.csv')

In [ ]:
association_rules_df

In [ ]:
P_A_list=[]
for i in range(len(association_rules_df['product_A'])):
  x = len(association_rules_df['product_A'][i])
  P_A_list.append(x)


P_B_list=[]
for j in range(len(association_rules_df['product_B'])):
  y = len(association_rules_df['product_B'][j])
  P_B_list.append(y)

association_rules_df['P_A_num']=P_A_list
association_rules_df['P_B_num']=P_B_list


In [ ]:
association_rules_df

## 功能：輸入商品找出關聯規則

In [ ]:
# 找出有此品項的規則
def find_rules(Prod):
  P=str('['+Prod+']')
  rules_list_test=[]
  for i in range(len(association_rules_df['product_A'])):
    if P in association_rules_df['product_A'][i]:
      rules_list_test.append(i)

  if rules_list_test !=[]:
    Result_df = association_rules_df.loc[rules_list_test]
    return Result_df
  else:
    print('This product NOT in current rules list \n Please select another product')
    # Prod=input()

In [ ]:
find_rules('JUMBO BAG PINK VINTAGE PAISLEY')

## 只針對1 對 1 的規則操作 

In [ ]:
association_rules_df_oneonone = association_rules_df[(association_rules_df['P_A_num'] == 1)&(association_rules_df['P_B_num'] == 1)]
association_rules_df_oneonone

處理[ ]問題
*   移除[ ]
*   取代原本的columns



In [ ]:
# remove [ ]
aaa = association_rules_df_oneonone['product_A']
Product_A=[]
for i in range(len(aaa)):
  s = str(aaa[i])[13:-4]
  Product_A.append(s)

bbb = association_rules_df_oneonone['product_B']
Product_B=[]
for i in range(len(aaa)):
  t = str(bbb[i])[13:-4]
  Product_B.append(t)

len(Product_A)
print('\n')
len(Product_B)

# replace product_A & B columns

support= association_rules_df_oneonone['Support']
Confidence= association_rules_df_oneonone['Confidence']
Lift= association_rules_df_oneonone['Lift']

dict_removement ={
    'Product_A' : Product_A,
    'Product_B' : Product_B,
    'Support' : support,
    'Confidence' : Confidence,
    'Lift' : Lift
}

association_rules_df_oneonone_done = pd.DataFrame(dict_removement)
association_rules_df_oneonone_done

檢查產品欄中的元素

In [ ]:
association_rules_df_oneonone_done['Product_A'].value_counts()
print('\n')
association_rules_df_oneonone_done['Product_B'].value_counts()

移除不要的商品(郵資......)

In [ ]:
# remove rules which have dotcom postage from Product_A & Product_B columns

tobe_remove = ['DOTCOM POSTAGE']
association_rules_df_oneonone_done = association_rules_df_oneonone_done[~association_rules_df_oneonone_done['Product_B'].isin(tobe_remove)]
association_rules_df_oneonone_done = association_rules_df_oneonone_done[~association_rules_df_oneonone_done['Product_A'].isin(tobe_remove)]

association_rules_df_oneonone_done.shape

In [ ]:
# reset index
association_rules_df_oneonone_done.reset_index(inplace=True, drop=True)
association_rules_df_oneonone_done.describe()

In [ ]:
# 太多了，設個條件(Support、Confidence平均值以上)，讓圖簡單一點
association_rules_df_oneonone_done_plot = association_rules_df_oneonone_done[(association_rules_df_oneonone_done['Support']>=0.015939)&(association_rules_df_oneonone_done['Confidence']>=0.631612)]
fig = px.sunburst(association_rules_df_oneonone_done_plot,
                path= ['Product_A','Product_B'],
                values= 'Support', 
                title='Sunburst plot of rules (part of one-on-one rules)',
                color= 'Confidence',
                color_continuous_scale='RdBu_r',
                hover_data=['Lift'],
                width= 800,
                height= 800
                  )

fig.show()

### 選擇 hot items❌
decide hot item list :
*   hot_item_list_done_quantity
*   hot_item_list_done_sales
*   hot_item_list_done_avg_Unit_Price

In [ ]:
# hot_item_list_done_quantity
# hot_item_list_done_sales
# hot_item_list_done_avg_Unit_Price

In [ ]:
# remove [ ] 
# do it again and just run this cell once
# aaa = hot_item_list_done_quantity
# hot_item_list_done_quantity=[]
# for i in range(len(aaa)):
#   s = str(aaa[i])[1:-1]
#   hot_item_list_done_quantity.append(s)

# aaa = hot_item_list_done_sales
# hot_item_list_done_sales=[]
# for i in range(len(aaa)):
#   s = str(aaa[i])[1:-1]
#   hot_item_list_done_sales.append(s)

# aaa = hot_item_list_done_avg_Unit_Price
# hot_item_list_done_avg_Unit_Price=[]
# for i in range(len(aaa)):
#   s = str(aaa[i])[1:-1]
#   hot_item_list_done_avg_Unit_Price.append(s)

# len(hot_item_list_done_quantity)
# print('\n')
# len(hot_item_list_done_sales)
# print('\n')

# len(hot_item_list_done_avg_Unit_Price)


In [ ]:
# if this cell report error,check 3 hot item lists,maybe the last cell ran twice or more.That cause items name error.

# hot_item_list_done = hot_item_list_done_quantity
# rules_quantity = association_rules_df[association_rules_df['Product_A'].isin(hot_item_list_done)]
# rules_quantity['Product_A'].value_counts()

# hot_item_list_done = hot_item_list_done_sales
# rules_sales = association_rules_df[association_rules_df['Product_A'].isin(hot_item_list_done)]
# rules_sales['Product_A'].value_counts()

# hot_item_list_done = hot_item_list_done_avg_Unit_Price
# rules_avg_Unit_Price = association_rules_df[association_rules_df['Product_A'].isin(hot_item_list_done)]
# rules_avg_Unit_Price['Product_A'].value_counts()

### hot item selection❌

using rules_quantity as rules

In [ ]:
# rules_quantity['Product_A'].value_counts()


In [ ]:
# rules_quantity.to_csv('rules_quantity.csv', index = False)

In [ ]:
# rules_quantity describe
# print('Support describe')
# rules_quantity['Support'].describe()
# print('\n=================================')
# print('Confidence describe')
# rules_quantity['Confidence'].describe()

In [ ]:
# rules_quantity_select = rules_quantity
# rules_quantity_select = rules_quantity[(rules_quantity['Support']>=0.012224)&(rules_quantity['Confidence']>=0.629253)]
# rules_quantity_select.sort_values(by='Lift', ascending=False)

### 處理 rules_quantity_select

In [ ]:
association_rules_df_oneonone_done

# 產出在Product_A & B 中的不重複產品清單
Product_A_list = list(association_rules_df_oneonone_done['Product_A'])
Product_B_list = list(association_rules_df_oneonone_done['Product_B'])
Product_list = list(set(Product_A_list).union(set(Product_B_list)))

Product_list
print('\n')
len(Product_list)

## 針對英國的預期收益圖表


*   英國資料
*   去掉退貨
*   有顧客編號
*   欄位整理





In [ ]:
data_want = data[data.loc[:,"Quantity"]>0] # 去掉退貨
data_want["Sales"] = data_want["UnitPrice"]*data_want["Quantity"] # 增加銷售額 
association_data = data_want[data_want['Description'].isin(Product_list)]
association_data = association_data[association_data['CustomerID'].notna()]   # CustomerID不是NaN
association_data = association_data[association_data['Country'] == 'United Kingdom']  # 留下英國
association_data = association_data.drop(['InvoiceNo', 'StockCode', 'InvoiceDate', 'Country'],axis=1)

association_data.head()
# association_data.to_csv('association_data.csv', index= False)

### 定義函式 關聯規則尋找視覺化❌

In [ ]:
# def rules_search(product):
#   target_product = association_rules_df[association_rules_df['Product_A'] == product]
#   fig = px.sunburst(target_product,
#                   path= ['Product_A','Product_B'],
#                   values= 'Support', 
#                   title='Sunburst plot of %s'%product ,
#                   color= 'Confidence',
#                   color_continuous_scale='RdBu',
#                   hover_data=['Lift'],
#                   width= 800,
#                   height= 800
#                   )

#   return fig.show()

In [ ]:
# rules_search("CHARLOTTE BAG SUKI DESIGN")

### 定義函式 生成成效預估資料集欄位

函式說明：


*   prod_A_list：customer ID list of product A buyers
*   prod_B_list：customer ID list of product B buyers
*   sales_list：customers buy product A but don't buy product B
*   unit_price：average unit price of product B
*   quantity：median of quantity of product B
*   confidence：confidence of association rules product A--->product B
*   revenue：anticipated revenue by unit_price * quantity







In [ ]:
def get_target(prod_A, prod_B):
  prod_A_list = list(association_data[association_data['Description'] == prod_A]['CustomerID'])
  prod_B_list = list(association_data[association_data['Description'] == prod_B]['CustomerID'])
  sales_list = list(set(prod_A_list)-set(prod_B_list))
  unit_price = association_data[association_data['Description'] == prod_B]['UnitPrice'].mean()
  quantity = association_data[association_data['Description'] == prod_B]['Quantity'].median()
  confidence = association_rules_df_oneonone_done[(association_rules_df_oneonone_done['Product_A'] == prod_A)&(association_rules_df_oneonone_done['Product_B'] == prod_B)]['Confidence'].mean()
  revenue = len(sales_list)*unit_price*quantity*confidence
  return [prod_A,prod_B,len(sales_list),unit_price,quantity,confidence,revenue]


In [ ]:
association_rules_df_oneonone_done

In [ ]:
result_list=[]
for i in range(len(association_rules_df_oneonone_done)):
  r= get_target(association_rules_df_oneonone_done['Product_A'][i] , association_rules_df_oneonone_done['Product_B'][i])
  result_list.append(r)

result_list[1]

# **sale list

In [ ]:
def get_target_2(prod_A, prod_B):
  prod_A_list = list(association_data[association_data['Description'] == prod_A]['CustomerID'])
  prod_B_list = list(association_data[association_data['Description'] == prod_B]['CustomerID'])
  sales_list = list(set(prod_A_list)-set(prod_B_list))
  unit_price = association_data[association_data['Description'] == prod_B]['UnitPrice'].mean()
  quantity = association_data[association_data['Description'] == prod_B]['Quantity'].median()
  confidence = association_rules_df_oneonone_done[(association_rules_df_oneonone_done['Product_A'] == prod_A)&(association_rules_df_oneonone_done['Product_B'] == prod_B)]['Confidence'].mean()
  revenue = len(sales_list)*unit_price*quantity*confidence
  return [prod_A,prod_B,len(sales_list),sales_list,unit_price,quantity,confidence,revenue]

In [ ]:
result_list_2=[]
for i in range(len(association_rules_df_oneonone_done)):
  r= get_target_2(association_rules_df_oneonone_done['Product_A'][i] , association_rules_df_oneonone_done['Product_B'][i])
  result_list_2.append(r)

result_list_2[1]

In [ ]:
revenue_table_2 = pd.DataFrame(
    data = result_list_2,
    columns=['Product_A','Product_B','Number of customers','sales list','unit_price','quantity','confidence','anticipated revenue']
    )

In [ ]:
revenue_table_2.head()

In [ ]:
revenue_table_2[revenue_table_2['Product_A']=='PAPER CHAIN KIT VINTAGE CHRISTMAS']

### 建立資料集 並視覺化

In [ ]:
revenue_table = pd.DataFrame(
    data = result_list,
    columns=['Product_A','Product_B','Number of customers','unit_price','quantity','confidence','anticipated revenue']
    )
revenue_table.describe()

In [ ]:
# 預期收益大於中位數的再畫圖
revenue_table_plot = revenue_table[revenue_table['anticipated revenue']>= 856.593159]
revenue_table_plot.describe()

In [ ]:
# 移除有0的再畫圖
revenue_table_plot = revenue_table_plot[~revenue_table_plot['anticipated revenue'].isin([0])]
revenue_table_plot.describe()

In [ ]:
fig = px.sunburst(revenue_table_plot,
                  path= ['Product_A','Product_B'],
                  values= 'Number of customers', 
                  title='Sunburst plot of Selected rules',
                  color= 'anticipated revenue',
                  color_continuous_scale='RdBu_r',
                  hover_data=['unit_price','confidence'],
                  width= 1000,
                  height= 800
                  )

fig.show()

In [ ]:
# 用圖形觀察
px.histogram(data_frame=rules_quantity, x='Support', nbins=20, title='Support histogram') 

px.histogram(data_frame=rules_quantity, x='Confidence', nbins=20, title='Confidence histogram') 

px.histogram(data_frame=rules_quantity, x='Lift', nbins=10, title='Lift histogram') 

# 筆記區


[切割時間說明](https://stackoverflow.com/questions/33151463/how-to-bin-time-in-a-pandas-dataframe)